# 교차 검증과 그리드 서치
#### 검증 세트가 필요한 이유를 이해하고 교차 검증에 대해 배웁니다.   그리드 서치와 랜덤 서치를 이용해 최적의 성능을 내는 하이퍼파라미터를 찾습니다.

지금까지 우리는 훈련 세트에서 모델을 훈련하고 테스트 세트에서 모델을 평가했다.

그런데 테스트 세트를 사용해 자꾸 성능을 확인하다 보면 점점 테스트 세트에 맞추게 되는 셈이다.

일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다.

마지막에 딱 한 번만 사용하는 것이 좋다.

그렇다면 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝을 어떻게 할 수 있을까?

게다가 결정 트리는 테스트해 볼 매개변수가 많다.

## 검증 세트

테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다.

그러면 간단한 방법은 훈련 세트를 또 나누는 것이다. 이 데이터를 **검증 세트**라고 부른다.

전체 데이터중 20%를 테스트 세트로 만들고 나머지 80%를 훈련 세트로 만들었는데 이 훈련세트 중에서 다시 20%를 떼어 내어 **검증 세트**로 만든다.

훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다.

그다음 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련한다.

마지막에 테스트 세트에서 최종 점수를 평가한다.

이전 절에 사용했던 데이터를 불러와서 검증 세트를 만들어보자. 먼저 판다스로 csv데이터를 읽는다.

In [3]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

# 그다음 class열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장한다.
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# 이제 훈련 세트와 테스트 세트로 나눌 차례이다. 훈련 세트와 입력 데이터와 타깃 데이터를 train_input과 train_target 배열에 저장한다.
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# 그 다음 train_input과 train_target을 다시 train_test_split() 함수에 넣어 
# 훈련세트 sub_input, sub_targetr과 검증세트 val_input, val_target을 만든다.
# test_size 매개변수를 0.2로 지정하여 train_input의 약 20%를 val_input으로 만든다.
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

# train_test_split() 함수를 2번 적용해서 훈련 세트와 검증 세트로 나눠준것 뿐이다. 훈련세트와 검증세트의 크기를 확인해보자.
print(sub_input.shape, val_input.shape)




(4157, 3) (1040, 3)


원래 5197개 였던 훈련세트가 4157개로 줄고 검증세트는 1040개가 되었다.

이제 sub_input, sub_target과 val_input, val_target을 사용해 모델을 만들고 평가해 보자

In [4]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


이 모델은 확실이 훈련세트에 과대 적합 되어있다. 매개변수를 바꿔서 더 좋은 모델을 찾아야 할 것 같다.

그 전에 검증 세트에 관해 좀 더 알아야 할 것이 있다.

## 교차 검증

**교차 검증**을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.

교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복한다. 이 점수를 평균하여 최종 검증 점수를 얻는다.

다음은 3-폴드 교차 검증 그림이다.

![사진](./fold.jpeg)

>3-폴드 교차 검증이 뭘까?  
훈련 세트를 세 부분으로 나눠서 교차 검증을 수행하는 것을 3-폴드 교차 검증이라고 한다.

각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.

사이킷 런에는 **cross_validate()** 라는 교차 검증 함수가 있다.

직접 검증 세트를 떼어내지 않고 훈련 세트 전체를 cross_validate() 함수에 전달한다.

In [5]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00814319, 0.00669456, 0.00688171, 0.00674534, 0.00648808]), 'score_time': array([0.00063348, 0.00039506, 0.00038838, 0.00039458, 0.00037837]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


cross_validate() 함수는 기본적으로 **5-폴드 교차 검증**을 수행한다.

교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다.

In [6]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있다.

우리는 train_test_split() 함수로 데이터를 섞은후 준비했기 때문에 따로 섞을 필요가 없지만

만약 교차검증을 할때 훈련 세트를 섞으로면 **분할기**를 지정해야 한다.

사이킷런의 분할기는 교차검증에서 폴드를 어떻게 나눌지 결정해준다.

회귀모델일경우 **KFold**분할기를 사용하고 분류 모델일경우 **StratifiedKFold**를 사용한다.

In [8]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

# 만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성한다.
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487
0.8574181117533719


이어서 결정 트리의 매겨변수 값을 바구가며 가장 좋은 성능이 나오는 모델을 찾아보자.

이때 테스트 세트를 사용하지 않고 교차 검증을 통해서 좋은 모델을 고르면 된다.

## 하이퍼파라미터 튜닝

머신러닝 모델이 학습하는 파라미터를 **모델 파라미터**라고 한다.

사용자가 지정해야만 하는 파라미터를 **하이퍼파라미터**라고 한다.

하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현된다.

하이퍼파라미터를 튜닝하는 작업은 라이브러리가 제공하는 기본값을 그대로 사용하는 방법이 있고 

검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔보는 방법이 있다.

매개변수를 바꿔가면서 모델을 훈련하고 교차 검증을 수행해야 한다.

그러면 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾아도 될까??

안된다. 두 매개변수를 동시에 바꿔가면서 최적의 값을 찾아야 한다.

매개변수가 많아지면 문제는 더 복잡해진다. 그래서 사이킷런에서 제공하는 **그리드 서치**를 사용한다.

사이킷런의 **GridSearchCV 클래스**는 하이퍼파라미터 탐색과 교차 검증을 한번에 수행해 준다.

기본 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값을 찾아보자

GridSearchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할값의 리스트를 딕셔너리로 만든다.

In [11]:
from sklearn.model_selection import GridSearchCV

# 여기서는 0.0001부터 0.0005까지 0.0001씩 증가하는 5개 값을 시도해보자.
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만든다.
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)

# gs 객체에 fit() 메서드를 호출한다.
# 이 메서드를 호출하면 그리드 서치 객체를 결정 트리 모델 min_impurity_decrease값을 바꿔가며 총 5번 실행한다.
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 한다.

사이킷런의 그리드 서치는 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.

이 모델은 gs객체의 best_estimator_ 속성에 저장되어 있다. 이모델을 일반 결정 트리 처럼 똑같이 사용가능하다.

In [13]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

# 그리드 서치로 찾은 최적의 매개변수는 best_params_속성에 저장되어 있다.
print(gs.best_params_)

0.9615162593804117
{'min_impurity_decrease': 0.0001}


여기서는 0.0001이 가장 좋은 값으로 선택되었다.

교차검증의 평균 점수는 cv_results_속성의 'mean_test_score'키에 저장되어 있다.

5번의 교차 검증으로 얻은 점수를 출력해 보자.

In [14]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


첫 번째 값이 가장 커 보인다.

넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.

이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다.

이 값이 최상의 검증 점수를 만든 매개변수 조합이다.

앞에서 출력한 gs.best_params_와 같은지 확인해보자

In [15]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


이 과정들을 정리해보자면

1. 먼저 탐색할 매개변수를 지정한다.
2. 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다.
3. 최상의 매개변수에서 전체 훈련세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.

조금 더 복잡한 매개변수 조합을 탐색해 보자.

결정 트리에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정한다.

여기에다 max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 골라보자.

In [17]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1), 'min_samples_split': range(2, 100, 10)}


넘파이 arange() 함수는 첫번째 매개변수 값에서 시작하여 두번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열을 만든다.

파이썬 range() 함수는 비슷한데 정수만 사용할 수 있다.

max_depth를 5에서 20까지 1씩 증가하면서 15개의 값을 만든다.

min_sample_split은 2에서 100까지 10씩 증가하면서 10개의 값을 만든다.

이 매개변수로 수행할 교차 검증 횟수는 9 * 15 * 10 = 1350개 이다.

5-폴드 교차 검증을 수행하므로 만들어지는 모델의 수는 6750개나 된다.

n_jobs 매개변수를 -1로 설정하고 그리드 서치를 실행해 보자.

In [23]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

# 최상의 매개변수 조합을 확인해 보자.
print(gs.best_params_)

# 최상의 교차 검증 점수도 확인해 보자.
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


GridSearchCV 클래스를 사용하니 원하는 매개변수 값을 나열하면 자동으로 교차 검증을 수행해서 최상의 매개변수를 찾을 수 있다.

하지만 앞에서 탐색할 매개변수의 간격을 0.0001 혹은 1로 설정했는데 이보다 더 좁거나 넓은 간격으로 시도해 볼 수 있지 않을까??

## 랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. 그리고 그리드 서치 수행시간이 오래 걸릴 수 있다.

이럴때 **랜덤 서치**를 사용하면 좋다.

랜덤서치는 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

싸이파이에서 2개의 확률 분포 클래스를 임포트 해보자.

In [31]:
# uniform 과 randint클래스는 모두 주어진 범위에서 고르게 값을 뽑는다. 이를 균등 분포에서 샘플링한다 라고 말한다.
# randint는 정숫값을 뽑고 uniform은 실수값을 뽑는다.
from scipy.stats import uniform, randint

# 0에서 10사이의 범위를 갖는 randint객체를 만들고 10개의 숫자를 샘플링 해보자.
rgen = randint(0,10)
print(rgen.rvs(10))

# 10개밖에 되지 않기 때문에 고르게 되는 것같지 않지만 숫자를 늘리면 쉽게 확인할 수 있다.
print(np.unique(rgen.rvs(1000), return_counts=True))

# 개수가 늘어나니 어느정도 고르게 추출된 것을 볼 수 있다. uniform클래스의 사용법도 동일히다.
ugen = uniform(0,1)
print(ugen.rvs(10))

[8 7 2 8 1 8 1 2 9 5]
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 86,  94, 104,  99,  86, 103,  91, 110, 100, 127]))
[0.86371138 0.26022075 0.28181756 0.38901892 0.0299211  0.21135593
 0.28226836 0.80242078 0.85276477 0.50878722]


난수 발생기랑 유사하게 생각하면 된다.

랜덤 서치에 randint와 uniform 클래스 객체를 넘겨주고 총 몇번을 샘플링해서 최적의 매개변수를 찾으라고 명령 할 수 있따.

min_samples_leaf 매개변수를 탐색 대상에 추가해보자. 이 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수이다.

탐색할 매개변수 범위는 다음과 같다.

In [32]:
params = {'min_impurity_decrease': uniform(0.0001, 0.0001),
         'max_depth': randint(20, 50),
         'min_samples_split': randint(2, 25),
         'min_samples_leaf': randint(1, 25)}

min_imputiry_decrease는 0.0001dptj 0.0001 사이의 실숫값을 샘플링한다.

max_depth는 20에서 50사이의 정수

min_samples_split은 2에서 25사이의 정수

min_samples_leaf는 1에서 25사이의 정수를 샘플링 한다.

샘플링 횟수는 RandomizedSearchCV의 n_iter 매개변수에 지정한다.

In [41]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                       n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

# 위 params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾는다.
# 먼저 최적의 매개변수 조합을 출력해보자.
print(gs.best_params_)

# 최고의 교차 검증 점수도 확인해보자.
print(np.max(gs.cv_results_['mean_test_score']))

# 최적의 모델은 이미 전체 훈련 세트로 후련되어 best_estimator_속성에 저장되어 있다.
# 이모델을 최종 모델로 결정하고 테스트 세트의 성능을 확인해보자.
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 47, 'min_impurity_decrease': 0.00015788648955075588, 'min_samples_leaf': 23, 'min_samples_split': 23}
0.8687702672688236
0.8569230769230769


테스트 세트는 검증 세트에 대한 점수보다 조금 작은 것이 일반적이다.

앞으로 수동으로 매개변수를 바꾸는 대신에 그리드 서치나 랜덤 서치를 사용해야겠다.

## 최적의 모델을 위한 하이퍼파라미터 탐색 <문제해결 과정>

선별하는 작업의 성능을 끌어올리기 위해 다양한 **하이퍼파라미터**를 시도해 봐야한다.

테스트 세트를 사용하면 결국 테스트 세트에 맞춰 모델을 훈련하는 효과를 만든다.

테스트 세트를 사용하지 않고 모델을 평가하려면 **검증 세트**가 필요하다.

검증세트는 훈련 세트 중 일부를 다시 덜어내어 만든다.

성능을 안정적으로 평가하기위해 검증세트를 여러번 반복해 모델을 평가할 수 있다. 이를 **교차 검증**이라 한다.

훈련 세트를 나눈 한 덩어리를 **폴드**라고 한다.

최종 검증 점수는 모든 폴드의 검증 점수를 평균하여 계산한다.

테스트 하고 싶은 매개변수 리스트를 만들어 이 과정을 자동화 하는 **그리드 서치**를 사용하면 편리하다.

매개변수 값이 수치형이고 연속적인 실숫값이라면 쌍파이의 확률분포 객체를 전단하여 매개변수 후보 값을 샘플링하여 교차 검증을 시도 할 수 있다.

다음 절에는 결정 트리를 확장하여 머신러닝 계를 재패한 **앙상블 모델**에 대해 알아보자